In [4]:
import xarray as xr
import numpy as np
import os
from metpy import calc as mpcalc

In [7]:
#The dropsonde file can be downloaded from: https://owncloud.gwdg.de/index.php/s/uy2eBvEI2pHRVKf?path=%2FLevel_3

if os.path.basename(os.path.expanduser("~")) == 'bfildier':
    inputdir = '/Users/bfildier/Data/EUREC4A/merged/sondes'
    radiosonde_path = os.path.join(inputdir,'all_radiosondes.nc')
    dropsonde_path = os.path.join(inputdir,'all_dropsondes.nc')
else:    
    inputdir = "/run/media/ludo/DATA/google-drive/Thèse/EUREC4a/github/Input/"
    dropsonde_path = os.path.join(inputdir, "Dropsondes", "EUREC4A_Dropsonde-RD41_Level_3A.nc")
    radiosonde_path = os.path.join(inputdir, "Radiosondes", "all_radiosondes.nc")



In [48]:
def prepare_dropsondes(dropsonde_path):
    dropsondes = xr.open_dataset(dropsonde_path)

    dropsondes = dropsondes.swap_dims({'sounding':'launch_time', 'obs':'height'}).reset_coords()
    dropsondes = dropsondes.drop_vars(["obs", "sounding","potential_temperature","precipitable_water","static_stability","low_height_flag",\
                                      "cloud_flag", "flight_height", "flight_lat", "flight_lon"])
    

    dropsondes["ascent_flag"] = (["launch_time"], np.zeros(len(dropsondes.launch_time.values)))

    return dropsondes

In [49]:
def prepare_radiosondes(radiosonde_path):

    radiosondes = xr.open_dataset(radiosonde_path)

    radiosondes = radiosondes.reset_coords(['lon','lat','time'])   
   
    radiosondes["specific_humidity"] = radiosondes["specific_humidity"]/1000
    

    radiosondes = radiosondes.rename({'gpsalt':'height', 'tdry':'temperature', "wspd":"wind_speed","pres":"pressure",\
                                 "lat":"latitude","lon":"longitude","rh":"relative_humidity","wdir":"wind_direction"})
    

    del radiosondes.attrs["title"]
    del radiosondes.attrs["doi"]
    del radiosondes.attrs["surface_altitude"]
    del radiosondes.attrs["featureType"]
    del radiosondes.attrs["Conventions"]
    del radiosondes.attrs["history"]
    del radiosondes.attrs["NCO"]
    del radiosondes.attrs["nco_openmp_thread_number"]

    length = len(radiosondes.launch_time.values)
    list_str = []
    for i in range(length):
        radiosonde = radiosondes.isel(launch_time = i)

        if (radiosonde.platform.values==1):
            list_str.append("BCO")
        elif (radiosonde.platform.values==2):
            list_str.append("MET")
        elif (radiosonde.platform.values==3):
            list_str.append("RHB")
        elif (radiosonde.platform.values==4):
            list_str.append("MER")
        else:
            list_str.append("ATL")     

    array = np.array(list_str, dtype=object)
    radiosondes["Platform"] = (["launch_time"], array)
    
    radiosondes = radiosondes.drop_vars(["dp","time","mr","ascent_rate","platform"])

        
    return radiosondes

In [50]:
dropsondes = prepare_dropsondes(dropsonde_path)
radiosondes = prepare_radiosondes(radiosonde_path)
print(dropsondes, radiosondes)

<xarray.Dataset>
Dimensions:            (height: 1001, launch_time: 1068)
Coordinates:
  * launch_time        (launch_time) datetime64[ns] 2020-01-19T16:55:14 ... 2020-02-11T09:43:11
  * height             (height) float64 0.0 10.0 20.0 ... 9.99e+03 1e+04
Data variables:
    latitude           (launch_time, height) float64 ...
    longitude          (launch_time, height) float64 ...
    pressure           (launch_time, height) float64 ...
    temperature        (launch_time, height) float64 ...
    relative_humidity  (launch_time, height) float64 ...
    wind_speed         (launch_time, height) float64 ...
    wind_direction     (launch_time, height) float64 ...
    u_wind             (launch_time, height) float64 ...
    v_wind             (launch_time, height) float64 ...
    specific_humidity  (launch_time, height) float64 ...
    Platform           (launch_time) object ...
    ascent_flag        (launch_time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    Title:       

In [55]:
all_sondes = xr.concat([dropsondes,radiosondes],dim='launch_time')

In [56]:
print(all_sondes)

<xarray.Dataset>
Dimensions:            (height: 3000, launch_time: 2522)
Coordinates:
  * height             (height) float64 0.0 10.0 20.0 ... 2.998e+04 2.999e+04
  * launch_time        (launch_time) datetime64[ns] 2020-01-19T16:55:14 ... 2020-02-16T04:14:33
Data variables:
    latitude           (launch_time, height) float64 nan 13.84 13.84 ... nan nan
    longitude          (launch_time, height) float64 nan -55.27 ... nan nan
    pressure           (launch_time, height) float64 nan 1.013e+03 ... nan nan
    temperature        (launch_time, height) float64 nan 25.89 25.79 ... nan nan
    relative_humidity  (launch_time, height) float64 nan 71.75 70.79 ... nan nan
    wind_speed         (launch_time, height) float64 nan 9.67 9.818 ... nan nan
    wind_direction     (launch_time, height) float64 nan 67.21 67.28 ... nan nan
    u_wind             (launch_time, height) float64 nan -8.916 ... nan nan
    v_wind             (launch_time, height) float64 nan -3.746 ... nan nan
    specific

In [57]:
all_sondes.to_netcdf(os.path.join(inputdir, "all_sondes.nc"))